In [383]:
import pandas as pd
pd.set_option("display.max_columns", None)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
from scipy import stats
from statsmodels.stats import weightstats as stests
import datetime as dt

In [384]:
df_az = pd.read_csv('Data without outliers.csv')

In [385]:
df_az.head()

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,dep_time,dep_delay,taxi_out,wheels_off,wheels_on,taxi_in,crs_arr_time,arr_time,arr_delay,cancelled,diverted,dup,crs_elapsed_time,actual_elapsed_time,air_time,flights,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,total_add_gtime,longest_add_gtime
0,2018-06-10,AA,AA_CODESHARE,AA,5393,OH,N580NN,5393,14685,SAV,"Savannah, GA",11057,CLT,"Charlotte, NC",1249,1245.0,-4.0,11.0,1256.0,1343.0,17.0,1403,1400.0,-3.0,0.0,0.0,N,74.0,75.0,47.0,1.0,213.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2019-05-08,WN,WN,WN,1065,WN,N267WN,1065,13232,MDW,"Chicago, IL",11066,CMH,"Columbus, OH",2255,2337.0,42.0,10.0,2347.0,129.0,3.0,105,132.0,27.0,0.0,0.0,N,70.0,55.0,42.0,1.0,283.0,27.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2018-11-09,UA,UA,UA,739,UA,N13110,739,13204,MCO,"Orlando, FL",11292,DEN,"Denver, CO",807,759.0,-8.0,13.0,812.0,1000.0,7.0,1005,1007.0,2.0,0.0,0.0,N,238.0,248.0,228.0,1.0,1546.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2018-01-27,AA,AA,AA,2283,AA,N972TW,2283,15624,VPS,"Valparaiso, FL",11298,DFW,"Dallas/Fort Worth, TX",1326,1317.0,-9.0,9.0,1326.0,1508.0,3.0,1543,1511.0,-32.0,0.0,0.0,N,137.0,114.0,102.0,1.0,641.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2019-11-05,DL,DL,DL,1678,DL,N321US,1678,12892,LAX,"Los Angeles, CA",11298,DFW,"Dallas/Fort Worth, TX",1755,1747.0,-8.0,13.0,1800.0,2229.0,10.0,2256,2239.0,-17.0,0.0,0.0,N,181.0,172.0,149.0,1.0,1235.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [386]:
df_az.columns 

Index(['fl_date', 'mkt_unique_carrier', 'branded_code_share', 'mkt_carrier',
       'mkt_carrier_fl_num', 'op_unique_carrier', 'tail_num',
       'op_carrier_fl_num', 'origin_airport_id', 'origin', 'origin_city_name',
       'dest_airport_id', 'dest', 'dest_city_name', 'crs_dep_time', 'dep_time',
       'dep_delay', 'taxi_out', 'wheels_off', 'wheels_on', 'taxi_in',
       'crs_arr_time', 'arr_time', 'arr_delay', 'cancelled', 'diverted', 'dup',
       'crs_elapsed_time', 'actual_elapsed_time', 'air_time', 'flights',
       'distance', 'carrier_delay', 'weather_delay', 'nas_delay',
       'security_delay', 'late_aircraft_delay', 'total_add_gtime',
       'longest_add_gtime'],
      dtype='object')

In [387]:
drop_columns = ['branded_code_share', 'mkt_carrier_fl_num', 'tail_num', 'op_carrier_fl_num', 'cancelled', 'diverted', 'flights', 'total_add_gtime', 'longest_add_gtime']

In [388]:
df_az = df_az.drop( axis =1 , labels = drop_columns)

In [389]:
len(df_az.columns)

30

In [390]:
df_az.head(3)

,fl_date,mkt_unique_carrier,mkt_carrier,op_unique_carrier,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,dep_time,dep_delay,taxi_out,wheels_off,wheels_on,taxi_in,crs_arr_time,arr_time,arr_delay,dup,crs_elapsed_time,actual_elapsed_time,air_time,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
0,2018-06-10,AA,AA,OH,14685,SAV,"Savannah, GA",11057,CLT,"Charlotte, NC",1249,1245.0,-4.0,11.0,1256.0,1343.0,17.0,1403,1400.0,-3.0,N,74.0,75.0,47.0,213.0,0.0,0.0,0.0,0.0,0.0
1,2019-05-08,WN,WN,WN,13232,MDW,"Chicago, IL",11066,CMH,"Columbus, OH",2255,2337.0,42.0,10.0,2347.0,129.0,3.0,105,132.0,27.0,N,70.0,55.0,42.0,283.0,27.0,0.0,0.0,0.0,0.0
2,2018-11-09,UA,UA,UA,13204,MCO,"Orlando, FL",11292,DEN,"Denver, CO",807,759.0,-8.0,13.0,812.0,1000.0,7.0,1005,1007.0,2.0,N,238.0,248.0,228.0,1546.0,0.0,0.0,0.0,0.0,0.0


In [391]:
def convert_time_to_string(x):
#     if np.isnan(x):
#         return '00:00'
    
    if x == 2400:
        return '23:59'
    
    string = str(int(x))
    
    if len(string) < 4:
        for _ in range(4-len(string)):
            string = '0' + string
    
    return string[:2] + ':' + string[2:]

In [392]:
#taxiout_depart_df = df_az.loc[(df_az['dep_time'].isna())]

In [393]:
df_az['dep_time'] = df_az['dep_time'].apply(lambda x: convert_time_to_string(x))

In [394]:
df_az['dep_time'] = pd.to_datetime(df_az['dep_time'], format ='%H:%M')

In [395]:
df_az['dep_hour'] = df_az['dep_time'].dt.hour

In [396]:
df_az['fl_date'] = pd.to_datetime(df_az['fl_date'])

In [397]:
df_az['fl_day'] = df_az['fl_date'].dt.day

In [398]:
df_az['fl_day']

0         10
1          8
2          9
3         27
4          5
          ..
154344     7
154345    26
154346    15
154347     6
154348     4
Name: fl_day, Length: 154349, dtype: int64

In [399]:
df_az['fl_month'] = df_az['fl_date'].dt.month

In [400]:
df_az = df_az.drop(axis = 1, labels = 'air_time')

In [401]:
df_az.to_csv("df_az_final.csv")

In [402]:
df_az.columns

Index(['fl_date', 'mkt_unique_carrier', 'mkt_carrier', 'op_unique_carrier',
       'origin_airport_id', 'origin', 'origin_city_name', 'dest_airport_id',
       'dest', 'dest_city_name', 'crs_dep_time', 'dep_time', 'dep_delay',
       'taxi_out', 'wheels_off', 'wheels_on', 'taxi_in', 'crs_arr_time',
       'arr_time', 'arr_delay', 'dup', 'crs_elapsed_time',
       'actual_elapsed_time', 'distance', 'carrier_delay', 'weather_delay',
       'nas_delay', 'security_delay', 'late_aircraft_delay', 'dep_hour',
       'fl_day', 'fl_month'],
      dtype='object')

In [403]:
df_az.head()

,fl_date,mkt_unique_carrier,mkt_carrier,op_unique_carrier,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,dep_time,dep_delay,taxi_out,wheels_off,wheels_on,taxi_in,crs_arr_time,arr_time,arr_delay,dup,crs_elapsed_time,actual_elapsed_time,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,dep_hour,fl_day,fl_month
0,2018-06-10,AA,AA,OH,14685,SAV,"Savannah, GA",11057,CLT,"Charlotte, NC",1249,1900-01-01 12:45:00,-4.0,11.0,1256.0,1343.0,17.0,1403,1400.0,-3.0,N,74.0,75.0,213.0,0.0,0.0,0.0,0.0,0.0,12,10,6
1,2019-05-08,WN,WN,WN,13232,MDW,"Chicago, IL",11066,CMH,"Columbus, OH",2255,1900-01-01 23:37:00,42.0,10.0,2347.0,129.0,3.0,105,132.0,27.0,N,70.0,55.0,283.0,27.0,0.0,0.0,0.0,0.0,23,8,5
2,2018-11-09,UA,UA,UA,13204,MCO,"Orlando, FL",11292,DEN,"Denver, CO",807,1900-01-01 07:59:00,-8.0,13.0,812.0,1000.0,7.0,1005,1007.0,2.0,N,238.0,248.0,1546.0,0.0,0.0,0.0,0.0,0.0,7,9,11
3,2018-01-27,AA,AA,AA,15624,VPS,"Valparaiso, FL",11298,DFW,"Dallas/Fort Worth, TX",1326,1900-01-01 13:17:00,-9.0,9.0,1326.0,1508.0,3.0,1543,1511.0,-32.0,N,137.0,114.0,641.0,0.0,0.0,0.0,0.0,0.0,13,27,1
4,2019-11-05,DL,DL,DL,12892,LAX,"Los Angeles, CA",11298,DFW,"Dallas/Fort Worth, TX",1755,1900-01-01 17:47:00,-8.0,13.0,1800.0,2229.0,10.0,2256,2239.0,-17.0,N,181.0,172.0,1235.0,0.0,0.0,0.0,0.0,0.0,17,5,11


In [404]:
s_1 = df_az.groupby('mkt_carrier')['carrier_delay'].mean()
s_1 = df_az.groupby('mkt_carrier')['weather_delay'].mean()
s_3 = df_az.groupby('mkt_carrier')['nas_delay'].mean()
s_4 = df_az.groupby('mkt_carrier')['security_delay'].mean()
s_5 = df_az.groupby('mkt_carrier')['late_aircraft_delay'].mean()

In [405]:
df_az = df_az.join(s_1, on = 'mkt_carrier', rsuffix='_mean')
df_az = df_az.join(s_1, on = 'mkt_carrier', rsuffix='_mean')
df_az = df_az.join(s_3, on = 'mkt_carrier', rsuffix='_mean')
df_az = df_az.join(s_4, on = 'mkt_carrier', rsuffix='_mean')
df_az = df_az.join(s_5, on = 'mkt_carrier', rsuffix='_mean')

In [406]:
df_az.head(2)

,fl_date,mkt_unique_carrier,mkt_carrier,op_unique_carrier,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,dep_time,dep_delay,taxi_out,wheels_off,wheels_on,taxi_in,crs_arr_time,arr_time,arr_delay,dup,crs_elapsed_time,actual_elapsed_time,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,dep_hour,fl_day,fl_month,weather_delay_mean,weather_delay_mean,nas_delay_mean,security_delay_mean,late_aircraft_delay_mean
0,2018-06-10,AA,AA,OH,14685,SAV,"Savannah, GA",11057,CLT,"Charlotte, NC",1249,1900-01-01 12:45:00,-4.0,11.0,1256.0,1343.0,17.0,1403,1400.0,-3.0,N,74.0,75.0,213.0,0.0,0.0,0.0,0.0,0.0,12,10,6,0.417663,0.417663,2.399724,0.014523,4.116784
1,2019-05-08,WN,WN,WN,13232,MDW,"Chicago, IL",11066,CMH,"Columbus, OH",2255,1900-01-01 23:37:00,42.0,10.0,2347.0,129.0,3.0,105,132.0,27.0,N,70.0,55.0,283.0,27.0,0.0,0.0,0.0,0.0,23,8,5,0.205839,0.205839,1.340548,0.013227,3.848458


In [407]:
# adding column for short, medium, long flight
df_az['flight_length'] = np.select (
    [ 
     (df_az['distance'] < 800),
     (df_az['distance'].between(800, 2200, inclusive=False)),
     (df_az['distance'] >=2200)
    ], 
    [
     'short',
     'medium',
     'long'
    ])

# df_az['flight_length'] = np.select (
#     [ 
#      (df_az['distance'] < 800),
#      (df_az['distance'].between(800, 2200, inclusive=False)),
#      (df_az['distance'] >=2200)
#     ], 
#     [
#      '1',
#      '2',
#      '3'
#     ])

In [408]:
# adding column for what part of day the flight depertured
# df_az['flight_time'] = np.select (
#     [ 
#      ((df_az['dep_hour'] > 0) & (df_az['dep_hour'] < 4)),
#         ((df_az['dep_hour'] >= 4) & (df_az['dep_hour'] < 7)),
#         ((df_az['dep_hour'] >= 7) & (df_az['dep_hour'] < 11)),
#         ((df_az['dep_hour'] >= 11) & (df_az['dep_hour'] < 15)), 
#         ((df_az['dep_hour'] >= 15) & (df_az['dep_hour'] < 19)),
#         ((df_az['dep_hour'] >= 19) & (df_az['dep_hour'] < 24))
#      ], 
#     [
#      'mid-night',
#      'early-morning',
#      'morning', 
#         'afternoon',
#         'evening',
#         'night'
#     ])

df_az['flight_time'] = np.select (
    [ 
     ((df_az['dep_hour'] > 0) & (df_az['dep_hour'] < 4)),
        ((df_az['dep_hour'] >= 4) & (df_az['dep_hour'] < 7)),
        ((df_az['dep_hour'] >= 7) & (df_az['dep_hour'] < 11)),
        ((df_az['dep_hour'] >= 11) & (df_az['dep_hour'] < 15)), 
        ((df_az['dep_hour'] >= 15) & (df_az['dep_hour'] < 19)),
        ((df_az['dep_hour'] >= 19) & (df_az['dep_hour'] < 24))
     ], 
    [
     'mid-night',
     'early-morning',
     'morning', 
        'afternoon',
        'evening',
        'night'
    ])

In [409]:
# adding column for what season the flight occured
df_az['flight_season'] = np.select (
    [ 
     ((df_az['fl_month'] >= 12) | (df_az['fl_month'] <= 2)),
        ((df_az['fl_month'] >= 3) & (df_az['fl_month'] <= 5)),
        ((df_az['fl_month'] >= 6) & (df_az['fl_month'] <= 8)),
        ((df_az['fl_month'] >= 9) & (df_az['fl_month'] <= 11))
     ], 
    ['winter',
     'spring',
     'summer', 
        'fall'])

In [410]:
df_az.head(1)

,fl_date,mkt_unique_carrier,mkt_carrier,op_unique_carrier,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,dep_time,dep_delay,taxi_out,wheels_off,wheels_on,taxi_in,crs_arr_time,arr_time,arr_delay,dup,crs_elapsed_time,actual_elapsed_time,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,dep_hour,fl_day,fl_month,weather_delay_mean,weather_delay_mean,nas_delay_mean,security_delay_mean,late_aircraft_delay_mean,flight_length,flight_time,flight_season
0,2018-06-10,AA,AA,OH,14685,SAV,"Savannah, GA",11057,CLT,"Charlotte, NC",1249,1900-01-01 12:45:00,-4.0,11.0,1256.0,1343.0,17.0,1403,1400.0,-3.0,N,74.0,75.0,213.0,0.0,0.0,0.0,0.0,0.0,12,10,6,0.417663,0.417663,2.399724,0.014523,4.116784,short,afternoon,summer


In [411]:
df_az.columns

Index(['fl_date', 'mkt_unique_carrier', 'mkt_carrier', 'op_unique_carrier',
       'origin_airport_id', 'origin', 'origin_city_name', 'dest_airport_id',
       'dest', 'dest_city_name', 'crs_dep_time', 'dep_time', 'dep_delay',
       'taxi_out', 'wheels_off', 'wheels_on', 'taxi_in', 'crs_arr_time',
       'arr_time', 'arr_delay', 'dup', 'crs_elapsed_time',
       'actual_elapsed_time', 'distance', 'carrier_delay', 'weather_delay',
       'nas_delay', 'security_delay', 'late_aircraft_delay', 'dep_hour',
       'fl_day', 'fl_month', 'weather_delay_mean', 'weather_delay_mean',
       'nas_delay_mean', 'security_delay_mean', 'late_aircraft_delay_mean',
       'flight_length', 'flight_time', 'flight_season'],
      dtype='object')

In [412]:
# categorical columns that will be kept for modelling
cat_col = ['flight_time', 'flight_season', 'flight_length']
# numerical columns that will be kept for modelling
num_col = ['distance', 'weather_delay_mean', 'nas_delay_mean', 'security_delay_mean', 'late_aircraft_delay_mean', 'arr_delay']

In [413]:
df = df_az.copy(deep=True)

In [414]:
df_cat = df[cat_col]

In [415]:
df_num = df[num_col]

In [416]:
df_num

,distance,weather_delay_mean,weather_delay_mean,nas_delay_mean,security_delay_mean,late_aircraft_delay_mean,arr_delay
0,213.0,0.417663,0.417663,2.399724,0.014523,4.116784,-3.0
1,283.0,0.205839,0.205839,1.340548,0.013227,3.848458,27.0
2,1546.0,0.393928,0.393928,3.393894,0.004749,4.136703,2.0
3,641.0,0.417663,0.417663,2.399724,0.014523,4.116784,-32.0
4,1235.0,0.366193,0.366193,2.031661,0.007455,2.796585,-17.0
...,...,...,...,...,...,...,...
154344,859.0,0.139386,0.139386,2.509660,0.014460,2.208605,-5.0
154345,224.0,0.139386,0.139386,2.509660,0.014460,2.208605,-1.0
154346,588.0,0.417663,0.417663,2.399724,0.014523,4.116784,-6.0
154347,118.0,0.417663,0.417663,2.399724,0.014523,4.116784,-15.0


In [417]:
df_cat

,flight_time,flight_season,flight_length
0,afternoon,summer,short
1,night,spring,short
2,morning,fall,medium
3,afternoon,winter,short
4,evening,fall,medium
...,...,...,...
154344,evening,spring,medium
154345,evening,spring,short
154346,morning,winter,short
154347,evening,fall,short


In [346]:
df_cat_dummies = pd.get_dummies(df_cat, drop_first = True)

In [347]:
df_cat_dummies

,flight_time_afternoon,flight_time_early-morning,flight_time_evening,flight_time_mid-night,flight_time_morning,flight_time_night,flight_season_spring,flight_season_summer,flight_season_winter
0,1,0,0,0,0,0,0,1,0
1,0,0,0,0,0,1,1,0,0
2,0,0,0,0,1,0,0,0,0
3,1,0,0,0,0,0,0,0,1
4,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
154344,0,0,1,0,0,0,1,0,0
154345,0,0,1,0,0,0,1,0,0
154346,0,0,0,0,1,0,0,0,1
154347,0,0,1,0,0,0,0,0,0


In [348]:
df_final = pd.concat([df_num, df_cat_dummies], axis = 1)

In [349]:
df_final

,distance,weather_delay_mean,weather_delay_mean,nas_delay_mean,security_delay_mean,late_aircraft_delay_mean,arr_delay,flight_time_afternoon,flight_time_early-morning,flight_time_evening,flight_time_mid-night,flight_time_morning,flight_time_night,flight_season_spring,flight_season_summer,flight_season_winter
0,213.0,0.417663,0.417663,2.399724,0.014523,4.116784,-3.0,1,0,0,0,0,0,0,1,0
1,283.0,0.205839,0.205839,1.340548,0.013227,3.848458,27.0,0,0,0,0,0,1,1,0,0
2,1546.0,0.393928,0.393928,3.393894,0.004749,4.136703,2.0,0,0,0,0,1,0,0,0,0
3,641.0,0.417663,0.417663,2.399724,0.014523,4.116784,-32.0,1,0,0,0,0,0,0,0,1
4,1235.0,0.366193,0.366193,2.031661,0.007455,2.796585,-17.0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154344,859.0,0.139386,0.139386,2.509660,0.014460,2.208605,-5.0,0,0,1,0,0,0,1,0,0
154345,224.0,0.139386,0.139386,2.509660,0.014460,2.208605,-1.0,0,0,1,0,0,0,1,0,0
154346,588.0,0.417663,0.417663,2.399724,0.014523,4.116784,-6.0,0,0,0,0,1,0,0,0,1
154347,118.0,0.417663,0.417663,2.399724,0.014523,4.116784,-15.0,0,0,1,0,0,0,0,0,0


In [350]:
df_final.corr()['arr_delay'].sort_values()

flight_time_early-morning   -0.112452
flight_time_morning         -0.087598
distance                    -0.021522
flight_time_afternoon       -0.017396
flight_season_spring        -0.012200
flight_season_winter        -0.004481
weather_delay_mean           0.002304
weather_delay_mean           0.002304
security_delay_mean          0.006785
nas_delay_mean               0.020507
flight_time_mid-night        0.038441
flight_season_summer         0.055176
flight_time_evening          0.057051
late_aircraft_delay_mean     0.062744
flight_time_night            0.134980
arr_delay                    1.000000
Name: arr_delay, dtype: float64

In [351]:
df_final.to_csv("df_modeling.csv")

In [352]:
df_final_sample = df_final.sample(frac =0.05)

In [353]:
df_final_sample

,distance,weather_delay_mean,weather_delay_mean,nas_delay_mean,security_delay_mean,late_aircraft_delay_mean,arr_delay,flight_time_afternoon,flight_time_early-morning,flight_time_evening,flight_time_mid-night,flight_time_morning,flight_time_night,flight_season_spring,flight_season_summer,flight_season_winter
137835,849.0,0.417663,0.417663,2.399724,0.014523,4.116784,-6.0,0,0,1,0,0,0,0,0,0
152581,683.0,0.417663,0.417663,2.399724,0.014523,4.116784,-25.0,0,0,1,0,0,0,1,0,0
129181,679.0,0.393928,0.393928,3.393894,0.004749,4.136703,-10.0,0,0,1,0,0,0,0,0,0
98113,258.0,0.366193,0.366193,2.031661,0.007455,2.796585,-15.0,0,0,1,0,0,0,0,0,1
18577,1205.0,0.708376,0.708376,1.699183,0.035240,4.271195,-14.0,1,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21240,725.0,0.205839,0.205839,1.340548,0.013227,3.848458,98.0,0,0,0,0,0,1,0,0,1
107445,192.0,0.393928,0.393928,3.393894,0.004749,4.136703,13.0,1,0,0,0,0,0,1,0,0
41476,846.0,0.393928,0.393928,3.393894,0.004749,4.136703,-7.0,1,0,0,0,0,0,0,0,1
38304,594.0,0.366193,0.366193,2.031661,0.007455,2.796585,-6.0,0,0,0,0,0,1,0,0,1


In [418]:
df_final_sample.to_csv("df_modeling_sample.csv")